<a href="https://colab.research.google.com/github/Akansha24786/mars/blob/master/Recommender_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Pre-req

In [1]:
!gdown --id 1dJVSSk8gFdjMs002N7oKDOthQKxFeuxT

Downloading...
From: https://drive.google.com/uc?id=1dJVSSk8gFdjMs002N7oKDOthQKxFeuxT
To: /content/filtered.csv
20.9MB [00:00, 44.9MB/s]


In [2]:
# Import libraries
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import spacy
import tensorflow_hub as hub
import matplotlib.pyplot as plt
from wordcloud import WordCloud
EN = spacy.load('en_core_web_sm')

In [3]:
import pandas as pd
df=pd.read_csv('filtered.csv')
df=df.replace('\r', ' ', regex=True)
df=df.replace('\n', ' ', regex=True)
df.head()

,Unnamed: 0,Issue key,Summary,Description,Custom field (Product L1 Name),Custom field (Product L2 Name),Custom field (Product L3 Name)
0,0,P10051779-113937,[Renesas R-CAR TOP] Huge number of repeaters a...,"Hello, We are currently working on a top le...",IC Compiler II,Pre-route Buffering,HFS
1,1,P10051779-114429,assign status on signal nets,"Hi, Assign statement is seen in Verilog out...",IC Compiler II,Pre-route Buffering,Other
2,2,P10051779-114676,initial_drc leaves inverters away from straps,dual-rail cells are away from straps after opt...,IC Compiler II,Pre-route Buffering,MV Buffering
3,3,P10051779-85889,initial_drc step of place_opt remove buffer an...,"Hi, My customer is using ICC2 18.06-SP4 and h...",IC Compiler II,Pre-route Buffering,HFS
4,4,P10051779-86232,PV REG: optimization log file increase 2X w/ l...,"Hi James, Regression Case: preroute_opt/drc/...",IC Compiler II,Pre-route Buffering,HFS


In [4]:
c=0
for i in range(len(df)):
    if isinstance(df['Summary'][i], float):
        df['Summary'][i]=' '
    if isinstance(df['Description'][i], float):
        df['Description'][i]=' '



for i in range(len(df)):
    if isinstance(df['Summary'][i], float):
        c+=1
print(c)


0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [5]:
df[df['Issue key']=='P10051779-166505']

,Unnamed: 0,Issue key,Summary,Description,Custom field (Product L1 Name),Custom field (Product L2 Name),Custom field (Product L3 Name)
198,198,P10051779-166505,initial_drc create detour buffer tree,place_opt -from initial_drc -to initial_drc cr...,IC Compiler II,Pre-route Buffering,HFS


In [6]:
# Make a dict having tag frequencies
tag_freq_dict = {}
for tag in df['Custom field (Product L2 Name)']:
        if tag not in tag_freq_dict:
            tag_freq_dict[tag] = 0
        else:
            tag_freq_dict[tag] += 1

In [7]:
print(tag_freq_dict)

{'Pre-route Buffering': 1198, 'Pre-route Opt': 3603, 'Placement - Coarse Placer': 1686, 'Chip Finish - Placement': 489, 'Relative Placement': 316, 'Magnet Placement': 15, 'Timing Analysis': 4645, 'Power Analysis': 1068, 'Custom field (Product L2 Name)': 0, nan: 13}


In [8]:
# Get most common tags
tags_to_use = 35
tag_freq_dict_sorted = dict(sorted(tag_freq_dict.items(), key=lambda x: x[1], reverse=True))
final_tags = list(tag_freq_dict_sorted.keys())[:tags_to_use]
len(final_tags)

10

In [9]:
import re
import nltk
import inflect
from nltk.corpus import stopwords


def tokenize_text(text):
    "Apply tokenization using spacy to docstrings."
    tokens = EN.tokenizer(text)
    return [token.text.lower() for token in tokens if not token.is_space]

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words

def normalize(words):
    words = to_lowercase(words)
    words = remove_punctuation(words)
    # words = remove_stopwords(words)
    return words

def tokenize_code(text):
    "A very basic procedure for tokenizing code strings."
    return RegexpTokenizer(r'\w+').tokenize(text)

def preprocess_text(text):
    return ' '.join(normalize(tokenize_text(text)))

In [10]:
import gensim 

W2V_SIZE = 300
W2V_WINDOW = 7
W2V_EPOCH = 64
W2V_MIN_COUNT = 20
import numpy as np

# Collect corpus for training word embeddings
# 2 3 4 5 6
# documents = [_text.split() for _text in np.array(df.Description)]
documents=[_text.split() for _text in np.array(df[df.keys()[2]])]
documents+=[_text.split() for _text in np.array(df[df.keys()[3]])]


w2v_model = gensim.models.word2vec.Word2Vec(size=W2V_SIZE, 
                                            window=W2V_WINDOW, 
                                            min_count=W2V_MIN_COUNT, 
                                            workers=8)
w2v_model.build_vocab(documents)
words = w2v_model.wv.vocab.keys()
vocab_size = len(words)
print("Vocab size", vocab_size)
# Train Word Embeddings
w2v_model.train(documents, total_examples=len(documents), epochs=W2V_EPOCH)
w2v_model.save('word2vec_embeddings.bin')


Vocab size 7837


In [11]:
# Load pre-trained embeddings
import gensim 

W2V_SIZE = 300
W2V_WINDOW = 7
W2V_EPOCH = 32
W2V_MIN_COUNT = 10
import numpy as np
w2v_model = gensim.models.word2vec.Word2Vec.load('word2vec_embeddings.bin')

In [12]:
# Tokenizing
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Dropout
from keras.utils.np_utils import to_categorical

# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 300
# This is fixed.
EMBEDDING_DIM = 300


tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['Description'])
word_index = tokenizer.word_index
vocab_size = len(word_index)
print('Found %s unique tokens.' % len(word_index))

# saving
import pickle
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)


Found 99134 unique tokens.


In [13]:
def question_to_vec(question, embeddings, dim=300):
    question_embedding = np.zeros(dim)
    valid_words = 0
    for word in question.split(' '):
        # print(word)
        # print(embeddings)
        if embeddings.__contains__(word):
            valid_words += 1
            question_embedding += embeddings[word]
    if valid_words > 0:
        return question_embedding/valid_words
    else:
        return question_embedding

In [14]:

all_title_embeddings = []
for title in df.Summary:
    all_title_embeddings.append(question_to_vec(title, w2v_model))
all_title_embeddings = np.array(all_title_embeddings)

embeddings = pd.DataFrame(data = all_title_embeddings)
embeddings.to_csv('title_embeddings.csv', index=False)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


In [15]:
all_title_embeddings = pd.read_csv('title_embeddings.csv').values

In [16]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

MAX_SEQUENCE_LENGTH = 300
import pickle
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [17]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

## Run

In [18]:
#@title Output


from IPython.display import HTML
import logging
from sklearn.metrics.pairwise import cosine_similarity

search_string = "Delay cells in IO paths in postroute causing setup violations" #@param {type:"string"}
search_string = ' '.join(normalize(tokenize_text(search_string)))
results_returned = "5" #@param {type:"string"}
search_vect = np.array([question_to_vec(search_string, w2v_model)])    # Vectorize the user query

# Calculate Cosine similarites for the query and all titles
cosine_similarities = pd.Series(cosine_similarity(search_vect, all_title_embeddings)[0])

# Custom Similarity Measure
cosine_similarities = cosine_similarities*(1)

output =""
for i,j in cosine_similarities.nlargest(int(results_returned)).iteritems():
    
    output += '<h2>' + df['Summary'][i] + '</h2>'
    output += '<h3> Issue Key: ' + df['Issue key'][i] + '</h3>'
    output += '<h3> Similarity Score: ' + str(j) + '</h3>'
    output +='<p style="font-family:verdana; font-size:110%;"> '
    for i in df.Description[i][:1000].split():
        if i.lower() in search_string:
            output += " <b>"+str(i)+"</b>"
        else:
            output += " "+str(i)
    output += "</p><hr>"
    
output = '<h3>Results:</h3>'+output
display(HTML(output))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':
